In [9]:
import pandas as pd 
import numpy as np
import os
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import altair as alt




## Function to get the boxscores with the starters

In [21]:
# sample_teams = ["Michigan","Duke","North Carolina","Kansas","Gonzaga","Purdue","Kentucky","Auburn","Tennessee","Baylor",'Texas Tech',"Villanova","Wisconsin"]
# path = 'C:\\Users\\Zach\\Desktop\\Capstone_offline_work\\src\\sample_player_boxscores\\'

### Players from 2019-2022 from the teams listed above
player_path = 'src\\sample_player_boxscores\\'




def boxscore_and_starters (player_path, year): 
    season_boxscore_df = pd.read_csv('{}{}_selected_player_boxscores.csv'.format(player_path,year),index_col = 0)
    ### split columns with a dash in between the statistic
    season_boxscore_df['FGM'], season_boxscore_df['FGA'] = zip(*season_boxscore_df['fg'].str.split('-').tolist())

    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["fg"])] 

    season_boxscore_df['3PM'], season_boxscore_df['3PA'] =zip(*season_boxscore_df['fg3'].str.split('-').tolist())
    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["fg3"])]

    season_boxscore_df['FTM'], season_boxscore_df['FTA'] =zip(*season_boxscore_df['ft'].str.split('-').tolist())
    season_boxscore_df = season_boxscore_df.loc[:,~ season_boxscore_df.columns.isin(["ft"])]


    ### Empty rows in 2022
    season_boxscore_df = season_boxscore_df[season_boxscore_df["min"]!='--']

    for var in ['FGM','FGA','3PM',"3PA" ,'FTM','FTA']:
        season_boxscore_df[var]=season_boxscore_df[var].astype(int)  
        season_boxscore_df.loc[:,"game_date"] = pd.to_datetime(season_boxscore_df.loc[:,"game_date"])

        #### Grabs the starting lineupes for each game 
        starters_season_boxscore_df =season_boxscore_df[season_boxscore_df.starter==True]
        starters_each_game_df = starters_season_boxscore_df.loc[:,["team_short_display_name","game_id","athlete_display_name"]].groupby(["team_short_display_name","game_id"]).agg(['unique']).reset_index()
        starters_each_game_df.columns = starters_each_game_df.columns.droplevel(1)
        starters_each_game_df= starters_each_game_df.rename(columns= {"athlete_display_name":"starting_lineup"})
        starters_each_game_df['starting_lineup']=starters_each_game_df.starting_lineup.apply(set)
    merged_lineups_and_boxscore_df = pd.merge(starters_each_game_df,season_boxscore_df, on = ["game_id","team_short_display_name"])
    return merged_lineups_and_boxscore_df

merged_lineups_and_boxscore_df = boxscore_and_starters(path,2019)
merged_lineups_and_boxscore_df


,team_short_display_name,game_id,starting_lineup,athlete_display_name,min,oreb,dreb,reb,ast,stl,...,team_color,season,season_type,game_date,FGM,FGA,3PM,3PA,FTM,FTA
0,Auburn,401082845,"{Bryce Brown, Samir Doughty, Jared Harper, Anf...",Chuma Okeke,32,2,4,6,2,3,...,03244d,2019,2,2018-12-20,3,8,1,6,0,0
1,Auburn,401082845,"{Bryce Brown, Samir Doughty, Jared Harper, Anf...",Anfernee McLemore,13,0,0,0,0,0,...,03244d,2019,2,2018-12-20,0,1,0,1,0,1
2,Auburn,401082845,"{Bryce Brown, Samir Doughty, Jared Harper, Anf...",Jared Harper,37,1,3,4,5,0,...,03244d,2019,2,2018-12-20,5,16,1,5,0,0
3,Auburn,401082845,"{Bryce Brown, Samir Doughty, Jared Harper, Anf...",Samir Doughty,20,1,0,1,2,1,...,03244d,2019,2,2018-12-20,3,6,1,3,0,0
4,Auburn,401082845,"{Bryce Brown, Samir Doughty, Jared Harper, Anf...",Bryce Brown,26,1,2,3,1,2,...,03244d,2019,2,2018-12-20,1,6,0,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,Wisconsin,401120737,"{Nate Reuvers, D'Mitrik Trice, Khalil Iverson,...",Tai Strickland,1,0,0,0,0,0,...,A00002,2019,2,2019-03-16,0,1,0,0,0,0
4402,Wisconsin,401120737,"{Nate Reuvers, D'Mitrik Trice, Khalil Iverson,...",Kobe King,26,1,2,3,0,0,...,A00002,2019,2,2019-03-16,5,10,1,3,2,3
4403,Wisconsin,401120737,"{Nate Reuvers, D'Mitrik Trice, Khalil Iverson,...",Walt McGrory,1,0,0,0,0,0,...,A00002,2019,2,2019-03-16,0,0,0,0,0,0
4404,Wisconsin,401120737,"{Nate Reuvers, D'Mitrik Trice, Khalil Iverson,...",Michael Ballard,1,0,0,0,0,0,...,A00002,2019,2,2019-03-16,0,0,0,0,0,0


Merge with Team boxscore to get results of each game - Missing Results from team boxscores folder - used largest_lead instead

In [26]:
def merge_team_box_score (merged_lineups_and_boxscore_df,path,year):
    team_boxscore_df = pd.read_csv('{}{}_selected_team_boxscores.csv'.format(path,year))
    ### Potentially use largest lead as outlier
    if "largest_lead" in team_boxscore_df.columns:
        columns = ["largest_lead","home_away","opponent_id","opponent_abbrev","game_id","team_abbreviation"]
    else:
        columns = ["home_away","opponent_id","opponent_abbrev","game_id","team_abbreviation"]
    simplified_team_box_score = team_boxscore_df.loc[:,columns]
    merged_df = pd.merge(merged_lineups_and_boxscore_df,simplified_team_box_score,on = ["game_id","team_abbreviation"])
    return merged_df 


In [29]:
year = 2019
for year in np.arange(2019,2023):
    print(year)
    players_df = boxscore_and_starters(player_path,year)
    merged_df = merge_team_box_score(players_df,teams_path,year)
    merged_df.to_csv("src\\team_player_boxscore_merged\\{}.csv".format(year))
    display(merged_df.head())


2019
2020
2021
2022


Starting Lineup Merged With BoxScores Averages - Further Analysis

In [30]:

# #### Merge the starters_each_game_df with season_boxscore_df (all players) on game_id how = (right - season_boxscores_df), then groupby team, lineup (as a set) and player to get the averages. 

# def get_averages_with_each_lineup(merged_lineups_and_boxscore_df):
#   #### Allows the startinglineup column to be included in the dataframe - makes the set hashable
#   merged_lineups_and_boxscore_df['starting_lineup']= [frozenset(s) for s in merged_lineups_and_boxscore_df['starting_lineup']]

#   #### Adjust columns starting with "pts" to include differnet columsn from the boxscore dataframe
#   averages_with_unique_LU = merged_lineups_and_boxscore_df.groupby(["team_short_display_name","starting_lineup","athlete_display_name"])[["pts",'reb', 'ast', 'stl', 'blk', 'to', 'pf',"min","season"]].mean().round(2).reset_index()

#   ### Calculate the number of games played by each player by counting the number of gamelogs (using the column points because if they played it will show as a 0 not a nan - some issues with min column)
#   ### Includes players who did not get any playing time/minutes 

#   games_with_line_up_df = merged_lineups_and_boxscore_df.groupby(["team_short_display_name","starting_lineup","athlete_display_name"])["pts"].count().reset_index().rename(columns = {"pts":"Player Games with Lineup"})
#   averages_with_unique_LU = pd.merge(averages_with_unique_LU,games_with_line_up_df,on = ["team_short_display_name","starting_lineup","athlete_display_name"])
#   averages_with_unique_LU["season"]=averages_with_unique_LU["season"].apply(int)

#   return averages_with_unique_LU
# avgs_with_lineup_df = get_averages_with_each_lineup(boxscore_and_starters(player_path,2021)).sort_values(by = 'Player Games with Lineup', ascending = False)
# avgs_with_lineup_df

In [14]:
# for yr in range(2005,2023):
#   print('Different unique number of games with a lineup in {}: {}'.format(yr, sorted(list(get_averages_with_each_lineup(boxscore_and_starters(player_path,yr))['Player Games with Lineup'].unique()))))

In [15]:
# avgs_with_lineup_df = get_averages_with_each_lineup(boxscore_and_starters(player_path,2015))
# avgs_with_lineup_df[avgs_with_lineup_df['Player Games with Lineup']==35]